# Preprocessing transcripts

In [12]:
%pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16664 sha256=1e0ee5d55a750e84b930610d6e1c482dd802e0a7a9c8b661256127fb63d727c8
  Stored in directory: /Users/mathiasbruun/Library/Caches/pip/wheels/60/08/06/19d2f21a2b05448da940fca0f0179e361fd90cec264fb379b6
Successfully built pandarallel
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Imports
import pickle
import pandas as pd
import numpy as np
import re
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [32]:
# Load transcripts from pickle
raw_transcripts = (
    pd.read_pickle('data/clean/subset_party_imputed_v2.p')
        .query('source == "parliament"')
        .drop(columns = ['index'])
    )  # TODO: change dataset after retrieval pipeline is cleaned up

In [33]:
print(raw_transcripts.shape)
raw_transcripts.head()

(92515, 6)


,doc,source,start_time,full_name,level,party
9,Tak. Danmarks kolonihistorie handler om militæ...,parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
10,"Jeg mener, at det er personen, der kan snakke ...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
11,"Det handler jo ikke om, hvad jeg synes. Det ha...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
12,"Jeg ved ikke, om det – når man snakker om at b...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
13,"Det er ikke noget, jeg bare mener; det er noge...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU


In [104]:
query_string = 'stærkt mandat'

[doc for doc in raw_transcripts['doc'] if query_string in doc][:30]

['jeg synes at statsministeren meget klart sagde fra over for et forslag som lå helt uden for hvad man kan acceptere fra dansk side og jeg er også glad for at når vi lægger tonediskussionen lidt til side lyder det som om vi stadig væk har venstre på en budgetrestriktiv linje og at vi kan blive enige om det sådan at ministeren rejser til bruxelles og de her vigtige forhandlinger med et stærkt mandat i ryggen',
 'tak så det blev simpelt hen udløst af statsministerens spørgsmål om hvor finansieringen skulle komme fra jeg synes ikke helt det var et svar på mit spørgsmål om hvorfor venstre brød med en års tradition for er det grundlæggende ikke vigtigt at vi giver regeringen et stærkt mandat i ryggen når regeringen skal ud at forhandle på vegne af danmark og danske interesser hvilket jeg ved at venstre også er interesseret i',
 'jamen det er svaret altså svaret er at statsministeren har nægtet finansministeren har nægtet udenrigsministeren har nægtet at svare på om et øget eu-budget kan fin

In [35]:
def remove_phrases(doc):
    """
    Remove phrases from a lowercased document.
    """
    
    # Remove names – requires uppercase and is thus done separately!
    name_pattern = r'(?:[Hh]r\. |[Ff]ru |[Ff]røken )[A-ZÆØÅ][a-zæøå]+(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)? '
    doc = re.sub(name_pattern, '', doc)

    # Consider removing party names and xxxministeren

    doc = doc.lower()

    # Remove procedural thank yous of different kinds
    politeness_pattern = r'^[\w\s,]*tak[\w\s,]*. '
    doc = re.sub(politeness_pattern, '', doc)
    
    return doc


In [36]:
test_doc = 'Tak. Fru Pernille Vermund brugte en del tid på at sværte fru Marie Krarup til, selv om fru Marie Krarup var så venlig at tilslutte sig beslutningsforslaget. Fru Pernille Vermunds anke var, at Dansk Folkeparti ikke havde gjort'

In [37]:
remove_phrases(test_doc)

'brugte en del tid på at sværte til, selv om var så venlig at tilslutte sig beslutningsforslaget. anke var, at dansk folkeparti ikke havde gjort'

In [38]:
import re
import string
import nltk
for dependency in ['punkt', 'wordnet', 'omw-1.4', 'stopwords', 'averaged_perceptron_tagger']:
    nltk.download(dependency)

def preproc_docs(text):
    #Lowercasing words
    text = text.lower()
    
    #Removing HTML tag
    text = re.sub(r'&amp', '', text)

    #Replace "&" with "and"
    text = re.sub(r'&','and', text)
    
    #Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation.replace('-',''))) #Taking hyphens out of punctuation to remove
    text = re.sub(r' - ','', text) #removing dash lines bounded by whitespace (and therefore not part of a word)
    text = re.sub(r'…', '', text)
    text = re.sub(r'[â€˜â€™â€œâ€â€”]','',text) #removing punctuation that is not captured by string.punctuation

    #Removing numbers
    text = re.sub(r'[0-9.]','', text)

    # Removing idiosynchratic characters in our data
    text = re.sub(r'-\n|\n-|\na-|\nb-|â€“|Â«|--|’', '', text)
    text = re.sub(r'- ', ' ', text)

    #Removing separators and superfluous whitespace
    text = text.strip()
    text = re.sub(r' +',' ',text)

    return text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [39]:
#raw_transcripts.doc = raw_transcripts.doc.parallel_apply(lambda x: remove_phrases(x))
raw_transcripts.doc = raw_transcripts.doc.parallel_apply(lambda x: preproc_docs(x))

In [40]:
raw_transcripts.doc

9         tak danmarks kolonihistorie handler om militær...
10        jeg mener at det er personen der kan snakke på...
11        det handler jo ikke om hvad jeg synes det hand...
12        jeg ved ikke om det – når man snakker om at bl...
13        det er ikke noget jeg bare mener det er noget ...
                                ...                        
349589    det burde jo så være kulturministeren jeg stil...
349590    jeg er meget glad for at det konservative folk...
349591    nej man kan ikke en til en sende en refunderin...
349592    jeg vil gerne starte med at kvittere for det g...
349593    ja det er det nye omprioriteringsbidrag som og...
Name: doc, Length: 92515, dtype: object

In [41]:
raw_transcripts.to_pickle('data/clean/preprocessed_docs.p')